## Creation of a model grid for a specific opening angle phi 

#### This grid is used as a model input unto the Monte-Carlo Simulation

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from math import asin,pi, sqrt
from plotUtilities import *
import matplotlib.patches as patches
import matplotlib
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,AutoMinorLocator)


In [ ]:
c       = 2.99792458e10 # en cm
vmax      = 0.9*c
vmin_dyn  = 0.08*c
vmax_wind = 0.08*c
vmin      = 0.025c
t         = 3600*24 # time equal to one day 
R         = t*vmax     #le rayon de la grille
nb_cell = 100  # numbers of cells in one direction
#R       = 7.77062*10**14
step    = 2*R/(nb_cell) 
phi     = pi/4   # ouverture angulaire de 45° % a l'axe de la fusion
alpha1  = 3
alpha2  = 6
#print(step/10**13,x_max,t,v)

# Defining the cells

In [ ]:
x = np.zeros(nb_cell)
for a in range(nb_cell):
    x[a] = R - a*step

y = np.zeros(nb_cell)
for a in range(nb_cell):
    y[a] = R - a*step
    
z = np.zeros(nb_cell)
for a in range(nb_cell):
    z[a] = R - a*step

In [ ]:
print(len(x))

In [ ]:
#alpha = pi/2 - phi 


def density(r,alpha):
    return  A*r**(-alpha)     # on prend A = 1
    

with open("grid_input_opang30.txt", "w+", encoding = 'utf-8') as model:
    model.write(f"{nb_cell} {R:e} {t} \n")
    for l in x:
        for m in y:
            for n in z: 
                r = sqrt((l-step/2)**2 + (m-step/2)**2 + (n-step/2)**2)      
                if r>R or r<(vmin*t):               #outside the sphere 
                    A = 1.
                    dens  = density(r,alpha1)
                    gamma = 0.4
                else:
                    A = 10**30 
                    if (vmin*t)<r<(vmin_dyn*t):  #in the wind outflow
                        dens  = density(r,alpha1)
                        gamma = 0.25
                    else:                         #in the dynamical outflow
                        dens = density(r,alpha2)
                        theta = asin((n-step/2)/r)
                        if abs(theta) < phi:     # red ejecta 
                            gamma = 0.2
                        else:
                            gamma = 0.4
                #else:
                    #if -theta < phi:    # red ejecta 
                     #   gamma = 0.2
                    #else:
                     #   gamma = 0.4
                model.write(f'{(l-step/2):e} {(m-step/2):e} {(n-step/2):e} {dens:e} {gamma}\n')

In [ ]:
asManyPlots(111, [Y,Z], Ye_tot, cmap='plasma', plotFlag=True, )

In [ ]:
data = 'grid_input2.txt'
x, y, z, dens, gamma = np.genfromtxt(data, skip_header=1, unpack=True)

In [ ]:
x1 = []
x2 = []
rho =[]
fract_electron =[]
for i in range(len(x)): 
    if x[i] == 7.770616e+12:
        x1.append(y[i])
        x2.append(z[i])
        rho.append(dens[i])
        fract_electron.append(gamma[i]) 

In [ ]:
print(len(x1), len(x2))

In [ ]:
Y, Z = np.meshgrid(np.sort(np.unique(x1)),np.sort(x2))
Gamma = np.copy(Y)

size1 = np.shape(Y)[0]
size2 = np.shape(Z)[0]

In [ ]:
for index1, yl, zl in zip(range(size1), Y, Z):
    for index2, valY, valZ in zip(range(size2), yl, zl):
        whereY       = x1 == valY
        whereZ       = x2 == valZ
        whereY_and_Z = np.logical_and(whereY,whereZ)
        position             = np.array(np.where(whereY_and_Z))[0][0]
        valF                 = fract_electron[position]
        Gamma[index1,index2] = valF
       

In [ ]:

ax = plt.subplot(111)
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
ax.tick_params(which='both', direction='in', labelsize=26)


dt = ax.contourf(Y, Z, Gamma, 200, cmap='coolwarm')
col = plt.colorbar(dt)

col.set_label(r'$\gamma_e$', size=26)
ax.set_xlabel(r'$y, \rm{cm}$', size=26)
ax.set_ylabel(r'$z, \rm{cm}$', size=26)
col.ax.tick_params(labelsize=22)
cont = plt.contour(Y, Z, Gamma, levels=[0.3,0.5,1], colors='black', linestyle="--")
cont.clabel(fontsize=10)
plt.savefig("ejecta_diagram.pdf", bbox_inches='tight')

In [ ]:
x = np.linspace(-5,5, 50)
def demi_cercle(x):
    y=[]
    for elt in x:
        a = sqrt(5**2 - elt**2)
        y.append(a)
    return y

def demi_cercle2(x):
    y=[]
    for elt in x:
        a = -sqrt(5**2 - elt**2)
        y.append(a)
    return y



y1 = demi_cercle(x) 
y2 = demi_cercle2(x)

In [ ]:
ax, lw = asManyPlots(111, [x,x], [y1,y2], plotFlag=[True, True], markerSize = [0,0], linestyle= ['-','-'], color=['black', 'black'], linewidth=4)

In [ ]:
def make_circle(r):
    t = np.arange(0, np.pi * 2.0, 0.01)
    #t = t.reshape((len(t), 1))
    x = r * np.cos(t)
    y = r * np.sin(t)
    return x,y
def make_circle2(r):
    t = np.arange(-np.pi/2, np.pi/2, 0.01)
    #t = t.reshape((len(t), 1))
    x = r * np.cos(t)
    y = r * np.sin(t)
    return x,y
x_circle,y_circle = make_circle(1)
x_circle2,y_circle2 = make_circle2(1)

In [ ]:
x_plus = np.arange(0.0, 1, 0.01)
x_neg = np.arange(-1, 0.0, 0.01)
def triangle1(phi,x):
    y = np.sin(phi)*x
    return y
def triangle2(phi,x):
    y = - np.sin(phi)*x
    return y

In [ ]:

a = int((len(x_circle)+1)/2)
#x  =  x_circle[:a]

#y  =  y_circle
y1 = triangle1(np.pi/6,x_plus)
y2 = triangle2(np.pi/6,x_plus)
y3 = triangle1(np.pi/6,x_neg)
y4 = triangle2(np.pi/6,x_neg)

In [ ]:
def make_circle(r):
    t = np.arange(0, np.pi/6, 0.01)
    #t = t.reshape((len(t), 1))
    x = r * np.cos(t)
    y = r * np.sin(t)
    return x,y

In [ ]:
vout_dyn = 0.3
ax_ratio_dyn=1
open_angle = 30

x_arc, y_arc = make_circle(0.1)

x = np.arange(-0.3, 0.4, 0.1)
y =np.zeros(len(x))
print(y)

In [ ]:
a = patches.Ellipse((0,0), vout_dyn*2,vout_dyn*2*ax_ratio_dyn,facecolor="steelblue",edgecolor="black", linewidth=1.5,zorder=1,alpha=1,label="Lanthanide-free")
plt.gca().add_patch(a)
a=patches.Wedge(0,(vout_dyn,vout_dyn*ax_ratio_dyn),-open_angle,open_angle,zorder=2,facecolor="indianred",alpha=1,edgecolor="black",label="Lanthanide-rich")
plt.gca().add_patch(a)
a=patches.Wedge(0,(-vout_dyn,-vout_dyn*ax_ratio_dyn),-open_angle,open_angle,zorder=2,facecolor="indianred",alpha=1,edgecolor="black")
plt.gca().add_patch(a)

plt.plot(x,y, color='black', linestyle='dotted')

plt.plot(x_arc, y_arc, color ='black', alpha = 0.8)

plt.gca().text(0.14, 0.035, r'$\phi$')



plt.title('Model of 2 ejecta components')
plt.gca().set_xlabel('velocity along equatorial plane (in c)')
plt.gca().set_ylabel('velocity along polar plane (in c)')
plt.gca().set_aspect(1)
plt.xticks(np.arange(-0.6,0.6,0.2))
plt.yticks(np.arange(-0.6,0.6,0.2))
plt.xlim(-0.5,0.5)
plt.ylim(-0.5,0.5)
plt.gca().xaxis.set_minor_locator(MultipleLocator(0.1))
plt.gca().yaxis.set_minor_locator(MultipleLocator(0.1))
plt.legend()
plt.savefig("model_ejecta.pdf", bbox_inches='tight')

In [ ]:
plt.plot(x_circle,y_circle)


#plt.plot(x, y1)
#plt.plot(x, y2)


#plt.fill_between(x_plus, y1, y2, facecolor='firebrick')
#plt.fill_between(x_neg, y3, y4, facecolor='firebrick')
plt.fill_between(x_plus+x_neg, y1, y_circle2, where=(y_circle2.all())>(y1.all()), facecolor='blue')

In [ ]:
np.shape(y_circle2)